In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [179]:
# Laste poll-of-polls data
df = pd.read_csv("../data/all_polls.csv", sep=",", encoding='ISO-8859-1')

In [180]:
# Convert to datetime and set the date to the end of the month
df["Dato"] = pd.to_datetime(df["Dato"])

In [181]:
# Sort values and set index
df = df.sort_values("Dato")
df.set_index("Dato", inplace=True)

In [182]:
df_en=df[["Ap","Høyre","Frp","SV","Sp","KrF","Venstre","MDG","Rødt"]]

In [183]:
df_en

,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt
Dato,,,,,,,,,
2008-01-09,32.7,14.1,23.2,7.1,6.4,7.8,6.0,0.0,0.0
2008-01-10,30.7,16.2,23.4,8.1,5.4,5.6,7.4,0.0,0.0
2008-01-18,29.5,17.7,25.6,6.4,5.0,6.4,6.1,0.0,0.0
2008-01-21,28.4,20.6,24.7,5.6,6.1,6.7,5.2,0.0,1.6
2008-01-21,30.7,18.3,24.2,7.1,5.4,6.2,5.4,0.0,1.1
...,...,...,...,...,...,...,...,...,...
2025-09-05,27.7,15.0,21.0,5.8,5.8,4.2,3.9,6.4,5.3
2025-09-05,26.4,14.3,20.4,6.0,6.2,4.2,4.3,7.2,6.9
2025-09-06,27.0,13.5,20.8,5.3,7.0,5.8,4.1,6.0,5.6


In [184]:
n_timesteps = 5   # past steps to look at
n_future = 12      # steps into the future we want to predict
n_features = df_en.shape[1]  
series_names = df_en.columns

In [185]:
n_features

9

In [186]:
def make_sequences_multistep(data, n_timesteps, n_future):
    X, y = [], []
    for i in range(len(data) - n_timesteps - n_future):
        X.append(data[i : i+n_timesteps])
        y.append(data[i+n_timesteps : i+n_timesteps+n_future].flatten())
    return np.array(X), np.array(y)

X, y = make_sequences_multistep(df_en.values, n_timesteps, n_future)
print("X shape:", X.shape, "y shape:", y.shape)

X shape: (1846, 5, 9) y shape: (1846, 108)


In [187]:
split = int(len(X) * 0.95)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [188]:
len(X_test)

93

In [ ]:
model = Sequential([
    LSTM(128, input_shape=(n_timesteps, n_features)),
    Dense(n_future * n_features)
])

model.compile(optimizer="adam", loss="mse")
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=1000, batch_size=32)

Epoch 1/1000


C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 160.6788 - val_loss: 76.4243
Epoch 2/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 70.9565 - val_loss: 28.4266
Epoch 3/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.4815 - val_loss: 13.3974
Epoch 4/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.2937 - val_loss: 11.7358
Epoch 5/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 14.6557 - val_loss: 13.2119
Epoch 6/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 13.2991 - val_loss: 14.6250
Epoch 7/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 12.8042 - val_loss: 15.4508
Epoch 8/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 12.4840 - val_loss: 15.5989
Epoch 9/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.7339 - val_loss: 15.2232
Epoch 10/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.9168 - val_loss: 15.3974
Epoch 11/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.1513 - val_loss: 15.1353
Epoch 12/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0

In [ ]:
y_true_one_step = y_test[:, 0, :]    # (185, 9)
y_pred_one_step = y_pred[:, 0, :]    # (185, 9)

In [ ]:
plt.figure(figsize=(15, 10))

for i in range(n_features):
    plt.subplot(3, 3, i+1)
    plt.plot(y_true_one_step[:, i], label="Actual")
    plt.plot(y_pred_one_step[:, i], label="Predicted")
    plt.title(series_names[i])
    plt.xlabel("Time step")
    plt.ylabel("Value")
    plt.legend()

plt.tight_layout()
plt.show()
